In [22]:
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm, trange
import time
import pickle
import datetime
import pandas as pd

In [95]:
def get_text_by_id(id_):
    url = 'https://archiveofourown.org/works/{}?view_full_work=true'
    headers = {'user-agent': 'bot (sj784@cornell.edu)'}
    
    r = requests.get(url.format(id_), headers=headers)
    soup = BeautifulSoup(r.text)
    
    assert soup != None
        
    stats = soup.find('dd', class_='stats')
    
    assert stats != None
    
    published_date = stats.find('dl', class_='stats').find('dd', class_='published').get_text() \
        if stats.find('dd', class_='published') else ''
    
    output = ''
    if soup.find_all('div', class_="userstuff module", role='article'):
        for chapter in soup.find_all('div', class_="userstuff module", role='article'):
            output += chapter.get_text()+'\n'
    
    elif soup.find('div', class_='userstuff'):
        output += soup.find('div', class_='userstuff').get_text()
    
    else:
        # print(f'can not find text for {id_}')
        raise AssertionError
        
    return output, published_date

def extract_info(id_, html):
    stats = html.find('dl', class_='stats')
    text, published_date = get_text_by_id(id_)
    return {
        'id': id_, # id
        'title': html.find('h4', class_='heading').get_text().split('\n')[1], # title
        'author': html.find('h4', class_='heading').get_text().split('\n')[-2],
        'rating': html.find_all('span', class_="text")[0].get_text(), # rating
        'fandoms': [tag.get_text() for tag in html.find('h5', class_='fandoms heading').find_all('a')], # fandoms
        'tags': [tag.get_text() for tag in html.find_all('li', class_='freeforms')], # tags
        'warning': html.find_all('span', class_="text")[1].get_text(), # warning
        'pairing': html.find_all('span', class_="text")[2].get_text(), # pairing,
        'comments': stats.find('dd', class_='comments').get_text() if stats.find('dd', class_='comments') else 0,
        'kudos': stats.find('dd', class_='kudos').get_text() if stats.find('dd', class_='kudos') else 0,
        'hits': stats.find('dd', class_='hits').get_text(),
        'relationships': [tag.get_text() for tag in html.find_all('li', class_='relationships')],
        'characters': [tag.get_text() for tag in html.find_all('li', class_='characters')],
        'summary': html.find('blockquote', class_='userstuff summary').get_text() if html.find('blockquote', class_='userstuff summary') else '',
        'text': text, # content
        'published_date': published_date,
        'timestamp': datetime.datetime.now()
    }
    

def is_valid(html):
    status = html.find_all('span', class_="text")[-1].get_text()
    
    stats = html.find('dl', class_='stats')
    words_count = stats.find('dd', class_='words').get_text().replace(',','')
    
    if not words_count:
        return False
    
    language = stats.find('dd', class_='language').get_text() 
    
    return status == 'Complete Work' and language == 'English' \
        and int(words_count) > 1000 and int(words_count) < 25000

    
def get_ids_by_tag(tag, num_required, existed, page_offset=0):
    url = 'https://archiveofourown.org/tags/{}/works?page={}'
    headers = {'user-agent': 'bot (sj784@cornell.edu)'}
    output = []
    
    page = 1 + page_offset
    delay = 300
    
    
    while True:
        page_count = 0

        r = requests.get(url.format(tag, page), headers=headers)
        soup = BeautifulSoup(r.text)
        
        try:
            error_cyle = 0
            
            works = soup.find_all('ol', class_='work index group')
            assert len(works) > 0
            
            for fanfic in works[0].find_all('li', role='article'):
                id_ = fanfic.find('h4', class_='heading').find('a', href=True)['href'].split('/')[-1]
                
                if id_ in existed:
                    continue
                if not is_valid(fanfic): 
                    continue
                
                output.append(extract_info(id_, fanfic))
                page_count += 1

            print(f'Scraped {page_count} from page {page}. Total: {len(output)}')
            
            if len(output) >= num_required:
                break 
            
            page += 1
        
        except AssertionError:
            print(f'Timed out. Will try again in {delay} sec')
            error_cyle += 1
            
            if error_cyle > 3:
                print('Waited too long. Exit')
                return output
            
            time.sleep(delay)
            page += 5
        

    
    print(f'Job complete. Scraped {len(output)} [{tag}] fanfics in total. Exit')
    return output

In [96]:
existed_df = pickle.load(open('data/ao3_db.p', 'rb'))
old_ids = set(existed_df['id'].tolist())
len(old_ids)

86

In [ ]:
scraped_ids = set()
data = []
for iter_ in range(5):
    print('Iteration', iter_+1)
    for tag in ['Angst', 'Fluff', 'Smut', 'Romance', 'Alternate%20Canon', 
                'Alternate%20Universe', 'Relationship(s)', 
                'Hurt*s*Comfort', 'Sexual%20Content']:
        old_ids |= scraped_ids
        data += get_ids_by_tag(tag, 100, old_ids, 1000 + iter_ * 100)
        scraped_ids = set([d['id'] for d in data])
    print(f'Scraped {len(data)} so far')

Iteration 1
Timed out. Will try again in 300 sec
Scraped 5 from page 1006. Total: 5
Scraped 7 from page 1007. Total: 12
Scraped 6 from page 1008. Total: 18
Scraped 6 from page 1009. Total: 24
Scraped 6 from page 1010. Total: 30
Scraped 11 from page 1011. Total: 41
Scraped 2 from page 1012. Total: 43
Scraped 2 from page 1013. Total: 45
Scraped 6 from page 1014. Total: 51
Scraped 3 from page 1015. Total: 54
Scraped 7 from page 1016. Total: 61
Scraped 5 from page 1017. Total: 66
Scraped 4 from page 1018. Total: 70
Scraped 5 from page 1019. Total: 75
Scraped 1 from page 1020. Total: 76
Scraped 8 from page 1021. Total: 84
Scraped 6 from page 1022. Total: 90
Scraped 6 from page 1023. Total: 96
Timed out. Will try again in 300 sec
Scraped 5 from page 1029. Total: 106
Job complete. Scraped 106 [Angst] fanfics in total. Exit
Scraped 8 from page 1001. Total: 8
Scraped 3 from page 1002. Total: 11
Scraped 0 from page 1003. Total: 11
Scraped 6 from page 1004. Total: 17
Scraped 5 from page 1005. Tot

In [78]:
def add_scraped_to_df(df, scraped_raw):
    scraped_df = pd.DataFrame(scraped_raw)
    mask = scraped_df['text'].str.len() > 1000
    scraped_df = scraped_df.loc[mask]
    return df.append(scraped_df, ignore_index=True)

In [79]:
existed_df = add_scraped_to_df(existed_df, data)

In [80]:
len(existed_df)

86

In [81]:
pickle.dump(existed_df, open('data/ao3_db.p', 'wb'))